[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/REPO_OWNER/comfy/blob/main/Kopie%20se%C5%A1itu%20comfyui_colab_with_manager.ipynb)

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [ ]:
# Checkpoints

### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

# SDXL ReVision
!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P ./models/clip_vision/

# SD1.5
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# SD2
!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora


# T2I-Adapter
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin


# ControlNet
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

# ControlNet SDXL
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/




In [ ]:
# ✅ ONE-SHOT FIX (deps + pin + ffmpeg) pro tvou sestavu
import os, sys, subprocess, shutil, glob

BASE = "/content/drive/MyDrive/ComfyUI" if os.path.isdir("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"
print("ComfyUI base:", BASE)

# A) vyřeš ofiko requirements
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "-r", f"{BASE}/requirements.txt"])

# B) chybějící knihovny z logu + pin pro controlnet_aux
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U",
    "pydantic-settings", "alembic", "av", "timm<1.0",
    "comfyui-workflow-templates", "comfyui-embedded-docs"
])

# C) ffmpeg pro video/audio nody (WAS apod.)
os.system("apt-get -y update && apt-get -y install -y ffmpeg")

print("✅ Hotovo. Teď nahoře znovu spusť buňku 'Run ComfyUI with cloudflare'.")


In [ ]:
# QUICK HEALTH CHECK
import importlib, sys
def ok(m):
    try:
        mod = importlib.import_module(m)
        v = getattr(mod, "__version__", "OK")
        print(f"✅ {m} {v}")
    except Exception as e:
        print(f"❌ {m} -> {e}")

for m in ["alembic","pydantic_settings","av","controlnet_aux","timm"]:
    ok(m)

import timm
from packaging import version
print("timm<1.0?", version.parse(timm.__version__) < version.parse("1.0"))


In [ ]:
# 🚧 Disable duplicitní ControlNet Aux složku
import os, shutil

BASE = "/content/drive/MyDrive/ComfyUI" if os.path.isdir("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"
cands = ["comfyui-controlnetaux", "comfyui_controlnet_aux"]
paths = [os.path.join(BASE, "custom_nodes", c) for c in cands]
existing = [p for p in paths if os.path.isdir(p)]
print("Nalezené ControlNetAux složky:", existing)

# necháme "comfyui_controlnet_aux", druhou (pokud existuje) vypneme
for p in existing:
    if os.path.basename(p) == "comfyui-controlnetaux":
        new = p + "_DISABLED"
        if not os.path.exists(new):
            os.rename(p, new)
            print("➡️ Vypnuto (přejmenováno):", p, "→", new)

print("✅ Hotovo. Po restartu bude aktivní jen comfyui_controlnet_aux.")


In [ ]:
# FIX: reinstall controlnet_aux čistě
import os, shutil, subprocess

CNODES = "/content/drive/MyDrive/ComfyUI/custom_nodes"
aux_dir = os.path.join(CNODES, "comfyui_controlnet_aux")

# smažeme starý rozbitý adresář
if os.path.isdir(aux_dir):
    shutil.rmtree(aux_dir)

# stáhneme čerstvě z gitu
subprocess.check_call(f"cd '{CNODES}' && git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git", shell=True)

print("✅ controlnet_aux reinstalován")


In [ ]:
# --- NUMPY/SCIPY/OPENCV COMPAT FIX pro controlnet_aux/DWpose ---
import sys, subprocess

def pipi(*args):
    print("pip", *args)
    subprocess.check_call([sys.executable, "-m", "pip", *args])

# Držíme NumPy na 1.26.4 a srazíme SciPy a OpenCV na ověřené verze
pins = [
    "numpy==1.26.4",           # ComfyUI-friendly
    "scipy==1.11.4",           # kompatibilní s NumPy 1.26.x a Python 3.11
    "opencv-python==4.8.1.78", # aby si to nevyžádalo NumPy 2.x
    "opencv-python-headless==4.8.1.78",
]
pipi("install", "--upgrade", "--force-reinstall", *pins)

print("\n✅ Pins applied. Doporučeno: soft restart Python procesu (viz další buňku) a pak znovu smoke-test.")


In [ ]:
import os, signal, time
print("🔄 Restartuji Python proces...")
os.kill(os.getpid(), signal.SIGKILL)


In [ ]:
# --- CONTROLNET_AUX SMOKE TEST ---
import importlib, sys, os, json

CNODES = "/content/drive/MyDrive/ComfyUI/custom_nodes"
sys.path.insert(0, CNODES)

mod = importlib.import_module("comfyui_controlnet_aux")
print("controlnet_aux import OK")

# Vezmeme pár typických tříd a zkontrolujeme, že INPUT_TYPES vrací slovníky (ne stringy)
names = [
    "controlaux_canny",
    "controlaux_midas",
    "controlaux_openpose",
    "controlaux_dwpose",
    "controlaux_lineart",
]

bad = []
for n in names:
    cls = getattr(mod, n, None)
    if cls is None:
        bad.append((n, "missing class"))
        continue
    try:
        it = cls.INPUT_TYPES()
        ok = isinstance(it, dict) and all(hasattr(v, "keys") for v in it.values())
        if not ok:
            bad.append((n, f"wrong INPUT_TYPES type: {type(it)}"))
    except Exception as e:
        bad.append((n, f"INPUT_TYPES error: {e}"))

if bad:
    print("\n❌ Problémové nody:")
    for n, m in bad:
        print("  -", n, "→", m)
else:
    print("\n✅ controlnet_aux vypadá zdravě.")


In [ ]:
# --- INSIGHTFACE SANITY CHECK ---
import os, sys, subprocess, importlib

def pipi(*args):
    print("pip", *args)
    subprocess.check_call([sys.executable, "-m", "pip", *args])

# 1) verze knihoven (jen vypíšeme nebo doinstalujeme, když chybí)
need = {
    "insightface": "0.7.3",
    "onnx": "1.16.2",
    # onnxruntime-gpu 1.18 funguje s CUDA 12.x na Colabu; když GPU provider není k dispozici, insightface si spadne na CPU provider
    "onnxruntime-gpu": "1.18.1",
}

for pkg, ver in need.items():
    try:
        m = importlib.import_module(pkg.replace("-", "_"))
        print(f"{pkg} present → OK ({getattr(m, '__version__', 'unknown')})")
    except Exception:
        pipi("install", "-U", f"{pkg}=={ver}")

# 2) rychlý běh FaceAnalysis (stáhne model do ~/.insightface, pokud není)
import insightface
import numpy as np
import cv2

print("insightface version:", insightface.__version__)
from insightface.app import FaceAnalysis

# preferujeme GPU, ale když není, insightface spadne na CPU provider
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]
app = FaceAnalysis(name="buffalo_l", providers=providers, allowed_modules=None)
app.prepare(ctx_id=0, det_size=(640, 640))

# syntetický mini-obrázek (žádné citlivé data), jde nám jen o průchod pipeline
img = np.full((640, 640, 3), 200, dtype=np.uint8)
cv2.circle(img, (320, 300), 120, (255, 255, 255), -1)  # „obličej“
faces = app.get(img)
print(f"FaceAnalysis OK, nalezeno obličejů: {len(faces)}; providers: {app.providers}")


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

In [ ]:
# ✅ ComfyUI INVENTORY (funguje s ComfyUI na Drive i v /content)
import os, sys, json, platform, subprocess, re
from pathlib import Path
from datetime import datetime

# 1) Najdi kořen ComfyUI
CANDIDATES = [
    "/content/drive/MyDrive/ComfyUI",
    "/content/ComfyUI",
    "/workspace/ComfyUI",
]
BASE = next((p for p in CANDIDATES if os.path.isdir(p)), None)
if BASE is None:
    raise SystemExit("❌ Nenašla jsem složku ComfyUI (zkontroluj cestu).")

USER_DIR = os.path.join(BASE, "user")
LOG_PATH = os.path.join(USER_DIR, "comfyui.log")

def git_commit(repo):
    try:
        if os.path.isdir(os.path.join(repo, ".git")):
            return subprocess.check_output(["git","rev-parse","HEAD"], cwd=repo, text=True).strip()
    except Exception:
        pass
    return "unknown/zip"

def git_remote(repo):
    try:
        if os.path.isdir(os.path.join(repo, ".git")):
            out = subprocess.check_output(["git","remote","-v"], cwd=repo, text=True).strip()
            fetches = [l for l in out.splitlines() if "(fetch)" in l]
            return fetches[0] if fetches else "n/a"
    except Exception:
        pass
    return "n/a"

# 2) Sys/Torch info
try:
    import torch
except Exception:
    torch = None

env = {
    "generated_at": datetime.now().isoformat(timespec="seconds"),
    "base_path": BASE,
    "platform": f"{platform.system()} {platform.release()} ({platform.machine()})",
    "python_version": sys.version.replace("\n"," "),
    "torch": getattr(torch, "__version__", "not installed"),
    "cuda_available": bool(torch and torch.cuda.is_available()),
    "cuda_version": getattr(getattr(torch, "version", None), "cuda", "n/a") if torch else "n/a",
    "device": (torch.cuda.get_device_name(0) if torch and torch.cuda.is_available() else "CPU"),
    "comfyui_commit": git_commit(BASE),
    "comfyui_remote": git_remote(BASE),
}

# pokus o čtení ComfyUI verze (z logu)
env["comfyui_version"] = None
env["frontend_version"] = None

log_text = ""
if os.path.isfile(LOG_PATH):
    try:
        with open(LOG_PATH, "r", encoding="utf-8", errors="ignore") as f:
            log_text = f.read()
        m = re.search(r"ComfyUI version:\s*([^\n]+)", log_text, re.IGNORECASE)
        if not m:
            m = re.search(r"ComfyUI Version:\s*([^\n]+)", log_text)
        if m: env["comfyui_version"] = m.group(1).strip()
        mf = re.search(r"ComfyUI frontend version:\s*([^\n]+)", log_text, re.IGNORECASE)
        if mf: env["frontend_version"] = mf.group(1).strip()
    except Exception:
        pass

# 3) Custom nodes (adresář + import status z logu)
cn_dir = os.path.join(BASE, "custom_nodes")
custom_nodes = sorted([n for n in os.listdir(cn_dir) if os.path.isdir(os.path.join(cn_dir, n))]) if os.path.isdir(cn_dir) else []
env["custom_nodes"] = custom_nodes

failed_nodes = []
missing_deps = set()

if log_text:
    # IMPORT FAILED hlášky
    for line in log_text.splitlines():
        if "IMPORT FAILED" in line:
            failed_nodes.append(line.strip())
        if "Please do a: pip install -r requirements.txt" in line:
            missing_deps.add("requirements.txt")
        if "No module named 'av'" in line: missing_deps.add("av")
        if "No module named 'alembic'" in line: missing_deps.add("alembic")
        if "pydantic-settings" in line or "pydantic_settings" in line: missing_deps.add("pydantic-settings")
        if "mutable default" in line and "timm" in log_text:
            missing_deps.add('timm<1.0 (pin)')

env["import_failures"] = failed_nodes
env["suspected_missing_deps"] = sorted(missing_deps)

# 4) Modely
models_dir = os.path.join(BASE, "models")
count = 0
by_ext = {}
if os.path.isdir(models_dir):
    for root, _, files in os.walk(models_dir):
        for f in files:
            count += 1
            ext = os.path.splitext(f)[1].lower()
            by_ext[ext] = by_ext.get(ext, 0) + 1
env["models_count"] = count
env["models_by_ext"] = dict(sorted(by_ext.items(), key=lambda x: (-x[1], x[0])))

# 5) Ulož report a vytiskni hezké summary + JSON
report_path = "/content/inventory_report.txt"
lines = []
lines.append("==== ComfyUI INVENTORY ====")
lines.append(f"Base: {env['base_path']}")
lines.append(f"ComfyUI commit: {env['comfyui_commit']}  | remote: {env['comfyui_remote']}")
lines.append(f"ComfyUI version: {env.get('comfyui_version') or 'n/a'} | Frontend: {env.get('frontend_version') or 'n/a'}")
lines.append(f"Python: {env['python_version']}")
lines.append(f"Torch: {env['torch']} | CUDA: {env['cuda_version']} | avail: {env['cuda_available']} | dev: {env['device']}")
lines.append(f"Custom nodes ({len(custom_nodes)}): {', '.join(custom_nodes) if custom_nodes else '(none)'}")
lines.append(f"Models: {env['models_count']} files | by ext: {env['models_by_ext']}")
if failed_nodes:
    lines.append("\n-- IMPORT FAILED lines from log --")
    lines.extend(failed_nodes[:30])  # zobraz max 30
if env["suspected_missing_deps"]:
    lines.append("\n-- Suspected missing deps / pin suggestions --")
    lines.append(", ".join(env["suspected_missing_deps"]))
lines.append("\n-- RAW JSON --")
lines.append(json.dumps(env, indent=2, ensure_ascii=False))

with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print("\n".join(lines[:1000]))
print(f"\n📄 Uloženo: {report_path}")


### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server